In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
df = pd.read_excel('../cashflows_2024.xlsx')

In [ ]:
df_obj = df.select_dtypes('object')

In [ ]:
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [ ]:
# df2 = df.melt(['Activity', 'Total'], var_name='Date', value_name='Measure')
# df2.loc[df2['Activity']=='Credit Card Receivable'].fillna(0)

In [ ]:
df3 = df.melt(['Activity'], var_name='Date', value_name='Measure').fillna(0)
# df3.loc[df3['Activity']=='Credit Card Receivable'].fillna(0)

In [ ]:
df3 = df3.drop(df3[df3['Measure'] == 0].index)
# df3

In [ ]:
df_totals = df3.groupby(['Activity']).sum('Measure')

In [ ]:
df_totals = df_totals.drop(df_totals[df_totals['Measure'] == 0].index)
# df_totals

In [ ]:
mask = df3['Measure'] < 0
df3['cash_inflow'] = df3['Measure'].mask(mask)
df3['cash_outflow'] = df3['Measure'].mask(~mask)
df4 = df3.drop(['Measure'], axis=1).fillna(0)

In [ ]:
df4['net_cash_flow'] = df4['cash_inflow'] - df4['cash_outflow'].abs()

In [ ]:
index_names = df4[df4['Date'] == 'Total'].index 
df4.drop(index_names, inplace = True) 

In [ ]:
# df4['date_of_sale'] = datetime.strptime(df4['Date'], '%B %d, %Y').strptime('%Y-%m-%d')
df4['Date'] = list(map(lambda x: datetime.strptime(x,'%b %d, %Y').strftime('%m/%d/%Y'), df4['Date']))

In [ ]:
df4['Date'] = pd.to_datetime(df4['Date'] )
# df4

In [ ]:
sf = pd.read_csv('../2024_sales_cogs_export.csv')

In [ ]:
sf['Date'] = pd.to_datetime(sf['date_of_sale'])
sf = sf.drop(['date_of_sale'], axis=1)

In [ ]:
sf.head()

In [ ]:
# df_merge = df4.merge(sf, how='cross')
df_merge = pd.merge(df4, sf, on='Date')

In [ ]:
df_merge.head(20)

In [ ]:
df4_totals = df4.groupby(['Activity']).sum('net_cash_flow')

In [ ]:
df4_totals

In [ ]:
df_totals = df_merge.groupby(['Activity']).sum('net_cash_flow')

In [ ]:
df_totals

In [ ]:
df_merge.to_csv('../cashflow_2024.csv', index=False)